In [ ]:
!pip install pandas

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('beers.csv')

In [14]:
df['embeddings'][0]

[0.026842087507247925,
 -0.004921723157167435,
 -0.0067475782707333565,
 -0.00449726264923811,
 0.014013941399753094,
 0.005774013232439756,
 -0.015347961336374283,
 -0.012396948412060738,
 0.00443662516772747,
 -0.02647826448082924,
 -0.006723996717482805,
 0.006373648531734943,
 -0.024376172572374344,
 -0.013986991718411446,
 -0.03134271875023842,
 0.008219716139137745,
 0.02805483341217041,
 -0.002090300666168332,
 0.022166281938552856,
 0.007269732188433409,
 0.012322835624217987,
 -0.0356547012925148,
 0.029806574806571007,
 -0.008253403007984161,
 -0.006845271214842796,
 0.010550880804657936,
 0.012174611911177635,
 -0.018918821588158607,
 -0.0002753941516857594,
 0.008165815845131874,
 0.026720814406871796,
 0.0147820133715868,
 -0.012423898093402386,
 0.0185280479490757,
 0.007559443358331919,
 -0.037460342049598694,
 0.0022722124122083187,
 -0.001061994000338018,
 0.01914789527654648,
 -0.025723667815327644,
 0.02184288389980793,
 -0.002540026791393757,
 0.00793000403791666,
 

In [3]:
def summary(beer):
    txt = f'''A cerveja {beer['name']} pertence a categoria {beer['category']}.
    
    Seus indicadores são: ABV:{beer['abv']}, IBU:{beer['ibu']}, EBC:{beer['ebc']}
    
    Algumas das comidas que vão combinar perfeitamente com essa cerveja são: {beer['foodPairing']}
    
    {beer['description']}
    
    Preparo da cerveja: {beer['brewersTips']}'''
    return txt

In [5]:
df['summary'] = df.apply(lambda df: summary(df), axis=1)

In [ ]:
!pip install openai

In [8]:
from openai import OpenAI
client = OpenAI(api_key="<your-key>")

In [9]:
def embedding(txt):
    res = client.embeddings.create(
        model='text-embedding-ada-002',
        input=txt
    )
    return res.data[0].embedding

In [10]:
df['embeddings'] = df['summary'].apply(embedding)

In [17]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [56]:
# Solicita o input ao usuário
question = input("Pergunta ao ibeer\n")

# Transforma o input em embedding
question_embedding = embedding(question)

# Compara esse embedding com os embeddings das cervejas utilizando a similaridade cossenoidal.
df['similarity'] = df['embeddings'].apply(lambda v: cosine_similarity(v, question_embedding))

# Retorna a cerveja que possui a maior similaridade
beer_summary = df.nlargest(1, 'similarity')['summary'].values
beer_summary

array(['A cerveja Stout Café Imperial pertence a categoria Stout.\n    \n    Adentre um café artesanal com nossa Stout Café Imperial. O aroma de grãos de café torrados preenche o ar, enquanto cada gole revela camadas de complexidade. A infusão de café imperial durante a maturação proporciona uma riqueza profunda de sabores de café, complementada pela suavidade da levedura. Feche os olhos e sinta-se em uma cafeteria aconchegante, explorando nuances de torrefação e cafeína.\n    \n    Seus indicadores são: ABV:7.8, IBU:35, EBC:40\n    \n    Algumas das comidas que vão combinar perfeitamente com essa cerveja são: Churrasco,Brownie de chocolate amargo,Queijo azul.\n    \n    O mestres cervejeiros que criaram esse cerveja fizeram uma descrição sobre seu preparo e o que a torna única: A infusão de café imperial durante a maturação proporciona uma experiência robusta, destacando a torrefação do malte e os aromas intensos de café, enquanto a levedura contribui para um final seco.',
       'A c

In [18]:
def send_question():
    question = input("Pergunta ao ibeer\n")
    question_embedding = embedding(question)
    df['similarity'] = df['embeddings'].apply(lambda v: cosine_similarity(v, question_embedding))
    context = df.nlargest(3, 'similarity')['summary'].values
    q = f'''context: {context}
    Q: {question}
    A: '''

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": '''
            Você é um especislita em cervejas artesanais, e irá atar na recomendação de cervejas para os clientes. 
            Responda pergunta abaixo somente se tiver 100% de certeza. 
            Você sempre deverá recomendar 3 cervejas para o usuário.
            Utilize um palavriado refinado e gentil, pense que você quer convercer o cliente a comprar aquela cerveja.
            Não fuja do contexto que será fornecido.
            Sempre mantenha a ordem das cervejas na qual foi informada no contexto.
            Sempre indique as comidas que mais combinam com cada cerveja.
            '''},
            {"role": "user", "content": q}
        ]
    )
    print(response.choices[0].message.content)

In [19]:
send_question()

Pergunta ao ibeer
 Quero uma cerveja preta para comer com um hamburguer


Claro, meu caro apreciador de cervejas! Para acompanhar um delicioso hambúrguer, permita-me apresentar a você três opções refinadas e requintadas.

1. Como primeira recomendação, sugiro a "Stout Café Imperial". Sua riqueza profunda de sabores de café vai se harmonizar perfeitamente com a suculência da carne do hambúrguer. A torrefação do malte e a infusão de café imperial proporcionarão uma experiência encorpada e sofisticada. 

2. Em segundo lugar, a "Double Chocolate Porter" é uma escolha extraordinária para acompanhar seu hambúrguer. A intensidade do cacau proporciona uma experiência decadente, combinando de forma sublime com a suculência da carne. A suavidade e riqueza dessa cerveja irão realçar os sabores do seu hamburguer.

3. Por fim, não posso deixar de mencionar a "Rye Pale Ale Picante". Apesar de não ser uma cerveja preta, suas notas picantes e terrosas provenientes do centeio malteado irão proporcionar um contraste exuberante com a carne do hambúrguer, elevando a experiência

In [20]:
df.nlargest(3, 'similarity')

,name,description,imageUrl,price,abv,ibu,ebc,category,foodPairing,brewersTips,summary,embeddings,similarity
9,Rye Pale Ale Picante,Entre em um jardim de especiarias com nossa Ry...,NaN,19.9,5.8,35,14,Pale Ale,"Hambúrguer de carne de porco,Queijo pepper jac...",A adição de centeio maltado proporciona à Pale...,A cerveja Rye Pale Ale Picante pertence a cate...,"[0.011999361217021942, -0.009249367751181126, ...",0.829971
2,Stout Café Imperial,Adentre um café artesanal com nossa Stout Café...,NaN,29.9,7.8,35,40,Stout,"Churrasco,Brownie de chocolate amargo,Queijo a...",A infusão de café imperial durante a maturação...,A cerveja Stout Café Imperial pertence a categ...,"[0.016071369871497154, -0.02415992133319378, -...",0.822183
10,Double Chocolate Porter,Delicie-se em uma chocolateria com nossa Doubl...,NaN,29.9,7.2,40,30,Porter,"Brownie de chocolate,Queijo azul envelhecido,C...",A adição de cacau em pó e nibs de cacau durant...,A cerveja Double Chocolate Porter pertence a c...,"[0.009299976751208305, -0.01400295551866293, -...",0.820612
